# Advent of Code

## 2019-012-007
## 2019 007

https://adventofcode.com/2019/day/7

In [1]:
from itertools import permutations

def get_value(program, parameter, mode):
    """Retrieve the value based on the parameter mode."""
    return program[parameter] if mode == 0 else parameter

def run_intcode(program, inputs):
    """Run the Intcode program with the given inputs."""
    position = 0
    output = None
    input_idx = 0

    while True:
        instruction = program[position]
        opcode = instruction % 100
        mode1 = (instruction // 100) % 10
        mode2 = (instruction // 1000) % 10
        mode3 = (instruction // 10000) % 10  # Not used for current opcodes

        if opcode == 99:
            break

        if opcode in (1, 2):  # Addition and Multiplication
            param1 = get_value(program, program[position + 1], mode1)
            param2 = get_value(program, program[position + 2], mode2)
            target = program[position + 3]
            if opcode == 1:
                program[target] = param1 + param2
            elif opcode == 2:
                program[target] = param1 * param2
            position += 4

        elif opcode == 3:  # Input
            target = program[position + 1]
            program[target] = inputs[input_idx]
            input_idx += 1
            position += 2

        elif opcode == 4:  # Output
            param1 = get_value(program, program[position + 1], mode1)
            output = param1
            position += 2

        elif opcode == 5:  # Jump-if-true
            param1 = get_value(program, program[position + 1], mode1)
            param2 = get_value(program, program[position + 2], mode2)
            if param1 != 0:
                position = param2
            else:
                position += 3

        elif opcode == 6:  # Jump-if-false
            param1 = get_value(program, program[position + 1], mode1)
            param2 = get_value(program, program[position + 2], mode2)
            if param1 == 0:
                position = param2
            else:
                position += 3

        elif opcode == 7:  # Less than
            param1 = get_value(program, program[position + 1], mode1)
            param2 = get_value(program, program[position + 2], mode2)
            target = program[position + 3]
            program[target] = 1 if param1 < param2 else 0
            position += 4

        elif opcode == 8:  # Equals
            param1 = get_value(program, program[position + 1], mode1)
            param2 = get_value(program, program[position + 2], mode2)
            target = program[position + 3]
            program[target] = 1 if param1 == param2 else 0
            position += 4

        else:
            raise ValueError(f"Unknown opcode {opcode} at position {position}")

    return output

def find_max_thruster_signal(program):
    """Find the maximum thruster signal."""
    max_signal = 0

    # Generate all permutations of phase settings [0, 1, 2, 3, 4]
    for phase_settings in permutations([0, 1, 2, 3, 4]):
        signal = 0
        for phase in phase_settings:
            # Copy the program for each amplifier
            program_copy = program[:]
            signal = run_intcode(program_copy, [phase, signal])
        max_signal = max(max_signal, signal)

    return max_signal

if __name__ == "__main__":
    # Read the Intcode program from input.txt
    with open("input.txt", "r") as file:
        program = list(map(int, file.read().strip().split(",")))

    max_signal = find_max_thruster_signal(program)
    print(f"Maximum thruster signal: {max_signal}")

Maximum thruster signal: 366376


In [2]:
from itertools import permutations
from collections import deque

class Amplifier:
    """Simulates a single amplifier with persistent Intcode state."""
    def __init__(self, program, phase):
        self.program = program[:]
        self.pointer = 0
        self.phase = phase
        self.inputs = deque([phase])
        self.halted = False

    def run(self, input_signal):
        """Runs the Intcode program until output is produced or halted."""
        self.inputs.append(input_signal)
        program = self.program

        while self.pointer < len(program):
            instruction = program[self.pointer]
            opcode = instruction % 100
            mode1 = (instruction // 100) % 10
            mode2 = (instruction // 1000) % 10
            mode3 = (instruction // 10000) % 10  # Not used

            def get_param(offset, mode):
                if mode == 0:  # Position mode
                    return program[program[self.pointer + offset]]
                elif mode == 1:  # Immediate mode
                    return program[self.pointer + offset]

            if opcode == 99:  # Halt
                self.halted = True
                return None

            elif opcode in {1, 2}:  # Add or multiply
                param1 = get_param(1, mode1)
                param2 = get_param(2, mode2)
                target = program[self.pointer + 3]
                program[target] = param1 + param2 if opcode == 1 else param1 * param2
                self.pointer += 4

            elif opcode == 3:  # Input
                if not self.inputs:
                    return None  # Wait for input
                target = program[self.pointer + 1]
                program[target] = self.inputs.popleft()
                self.pointer += 2

            elif opcode == 4:  # Output
                output = get_param(1, mode1)
                self.pointer += 2
                return output

            elif opcode == 5:  # Jump-if-true
                param1 = get_param(1, mode1)
                param2 = get_param(2, mode2)
                self.pointer = param2 if param1 != 0 else self.pointer + 3

            elif opcode == 6:  # Jump-if-false
                param1 = get_param(1, mode1)
                param2 = get_param(2, mode2)
                self.pointer = param2 if param1 == 0 else self.pointer + 3

            elif opcode == 7:  # Less than
                param1 = get_param(1, mode1)
                param2 = get_param(2, mode2)
                target = program[self.pointer + 3]
                program[target] = 1 if param1 < param2 else 0
                self.pointer += 4

            elif opcode == 8:  # Equals
                param1 = get_param(1, mode1)
                param2 = get_param(2, mode2)
                target = program[self.pointer + 3]
                program[target] = 1 if param1 == param2 else 0
                self.pointer += 4

            else:
                raise ValueError(f"Unknown opcode {opcode} at position {self.pointer}")

def find_max_feedback_signal(program):
    """Find the maximum thruster signal in feedback loop configuration."""
    max_signal = 0

    # Generate all permutations of phase settings [5, 6, 7, 8, 9]
    for phase_settings in permutations([5, 6, 7, 8, 9]):
        # Initialize amplifiers
        amplifiers = [Amplifier(program, phase) for phase in phase_settings]
        
        signal = 0
        while True:
            for amp in amplifiers:
                output = amp.run(signal)
                if output is not None:
                    signal = output
            # If the last amplifier halts, break
            if amplifiers[-1].halted:
                break
        
        max_signal = max(max_signal, signal)

    return max_signal

if __name__ == "__main__":
    # Read the Intcode program from input.txt
    with open("input.txt", "r") as file:
        program = list(map(int, file.read().strip().split(",")))

    max_signal = find_max_feedback_signal(program)
    print(f"Maximum thruster signal: {max_signal}")

Maximum thruster signal: 21596786
